In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
tf.random.set_seed(42)

pd.set_option('display.max_columns', 500) 

# Load Data: DB1(2020.5.17).xlsx


In [2]:
#data_jaun = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'digestive(jaundice)')
#data_hemate = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'hematemesis(digestive)')
#data_Folie_Nerve = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'Folie_Nerve')
#data_musculoskeletal = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'musculoskeletal')
#data_kidney = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'kidney')
#data_respiratory = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'respiratory')
#data_circulator1 = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'circulator1')
#data_circulator2 = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'Circulator2')
#data_para = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'Para_Female_Child')
#data_digestive = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'digestive')

In [5]:
data_jaun = pd.read_excel('DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'digestive(jaundice)')
data_hemate = pd.read_excel('DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'hematemesis(digestive)')
#data_Folie_Nerve = pd.read_excel('DB1(2020.5.17).xlsx', sheet_name = 'Folie_Nerve')
data_musculoskeletal = pd.read_excel('DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'musculoskeletal')
data_kidney = pd.read_excel('DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'kidney')
data_respiratory = pd.read_excel('DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'respiratory')
data_circulator1 = pd.read_excel('DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'circulator1')
data_circulator2 = pd.read_excel('DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'Circulator2')
data_para = pd.read_excel('DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'Para_Female_Child')
data_digestive = pd.read_excel('DB1(2020.5.17)_add_less_5.xlsx', sheet_name = 'digestive')
#others = pd.read_excel('결측치db.xls', sheet_name='Sheet2')

# 1. Preprocessing

## 0) Select columns and fill diseases

In [6]:
def organize_data(df):
  df.columns = df.iloc[0]
  df = df[1:]
  df = df.iloc[:, 3:]
  df = df[['level2', 'Chief complaint', 'Onset', 'Location']]
  #df = df[['level2', 'Chief complaint', 'Onset']]
  df['level2'] = df['level2'].fillna(method='ffill')

  return df

In [7]:
data_digestive = organize_data(data_digestive)
data_musculoskeletal = organize_data(data_musculoskeletal)
data_kidney = organize_data(data_kidney)
data_respiratory = organize_data(data_respiratory)
data_para = organize_data(data_para)
#data_Folie_Nerve = organize_data(data_Folie_Nerve)
data_circulator1 = organize_data(data_circulator1)
data_hemate = organize_data(data_hemate)
data_jaun = organize_data(data_jaun)

data_circulator2 = data_circulator2[['챕터', 'Chief complaint', 'Onset', 'Location']]
#data_circulator2 = data_circulator2[['챕터', 'Chief complaint', 'Onset']]
data_circulator2['챕터'] = data_circulator2['챕터'].fillna(method='ffill')

In [8]:
print(data_digestive.shape)
print(data_jaun.shape)
print(data_hemate.shape)
print(data_musculoskeletal.shape)
print(data_kidney.shape)
print(data_respiratory.shape)
print(data_para.shape)
#print(data_Folie_Nerve.shape)
print(data_circulator1.shape)
print(data_circulator2.shape)

(421, 4)
(440, 4)
(360, 4)
(205, 4)
(246, 4)
(205, 4)
(355, 4)
(305, 4)
(218, 4)


In [9]:
data_circulator2.columns = data_circulator1.columns

data = pd.concat([data_digestive, data_musculoskeletal, data_kidney,
                  data_respiratory, data_circulator1, #data_Folie_Nerve,
                  data_circulator2, data_para, data_jaun, data_hemate], axis=0)
data.sample()

,level2,Chief complaint,Onset,Location
204,Dysuria,"""소변보는 게 불편해요""",3시간 전,-


In [10]:
data = data.dropna(thresh=2)
data.shape

(2363, 4)

## 1) Check Target Data, Disease

In [11]:
data['level2'].value_counts()

Jaundice                            495
Hematemesis                         400
Emmeniopathy/Dysmenorrhea           100
Acute Abdominal Pain                 95
Arthrodynia                          75
Mastalgia/Breast Lump                70
Chest Pain                           66
Blackout                             65
Emesis                               60
Cough                                60
Red Urine                            60
Dysuria                              60
Dyspnea                              60
Dyspepsia/Chronic Abdominal Pain     55
Palpitation                          55
Dyslipidemia                         53
Hemoptysis                           50
Cervicodynia                         50
Skin Rash                            50
Oliguresis                           50
Diarrhea                             45
Hypertension                         43
Hematochezia                         40
Polyuria                             40
Rhinorrhea/Nasal Stuffiness          40


In [12]:
data = data[data['level2'] != 'Dyslipidemia'] # Drop 이상지질혈증
data.reset_index(inplace=True, drop=True)

In [13]:
len(data['level2'].unique())

28

### Organize differently expressed diseases to the same expression diseases

In [14]:
level2_kor = pd.read_excel('ver6_new_words.xlsx', sheet_name = 'level2')
level2_kor = level2_kor[['Level 2', '번역']]
level2_kor = level2_kor.dropna()

level2_kor.columns = ['level2', '번역']
level2_kor.head()

,level2,번역
0,Acute Abdominal Pain,급성 복통
1,Dyspepsia/Chronic Abdominal Pain,소화불량/만성 복통
2,Hematemesis,토혈
3,Hematochezia,혈변
4,Emesis,구토


In [15]:
print(level2_kor.shape)
print(data.shape)

(41, 2)
(2310, 4)


In [16]:
data = pd.merge(data, level2_kor, how='left',on='level2')
#data2.reset_index(drop=True, inplace=True)

data['번역'].fillna(data['level2'], inplace=True)
data['level2'] = data['번역']
data.drop('번역', axis=1, inplace=True)
data.sample()

,level2,Chief complaint,Onset,Location
628,다뇨증,"""소변량이 많아졌어요""",이년전,-


In [17]:
len(data['level2'].unique())

27

In [18]:
a = pd.DataFrame(data['level2'].value_counts().reset_index().values, columns=["disease", "counts"])
a['disease']

0             황달
1             토혈
2           호흡곤란
3       월경이상/월경통
4          급성 복통
5     목 통증/허리 통증
6          관절 통증
7       유방통/유방덩이
8           가슴통증
9             실신
10        붉은색 소변
11         배뇨 이상
12            기침
13            구토
14          두근거림
15    소화불량/만성 복통
16            객혈
17          피부발진
18            핍뇨
19            설사
20           고혈압
21            혈변
22           다뇨증
23        콧물/코막힘
24         소변찔끔증
25         질 분비물
26            변비
Name: disease, dtype: object

## 2) Change Nan

In [19]:
data = data.fillna('-')

In [20]:
def to_nan(x):
  if(x == '-'):
    x = ''
  elif(x == '아니오'):
    x = ''
  elif(x == '아뇨'):
    x = ''
  elif(x == '몰라요'):
    x = ''
  elif(x == '모릅니다'):
    x = ''
  elif(x == '모름'):
    x = ''
  elif(x == '아뇨'):
    x = ''
  elif(x == '아뇨'):
    x = ''
  elif(x == '없습니다'):
    x = ''
  elif(x == '없어요'):
    x = ''
  elif(x == '없음'):
    x = ''
  elif(x == '.'):
    x = ''
  return x

In [21]:
for i in range(len(data.columns)):
    data[data.columns[i]] = data.apply(lambda x : to_nan(x[data.columns[i]]) , axis = 1 )

## 3) Make a sentence

In [22]:
data['All'] = data['Chief complaint'] + '. ' + data['Onset'] + '. ' + data['Location']
data.sample()

,level2,Chief complaint,Onset,Location,All
1957,황달,눈이 노랗게 됐어요,3일전,"얼굴, 눈","눈이 노랗게 됐어요. 3일전. 얼굴, 눈"


## 4) Check spelling

In [23]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-falgwwcq
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-falgwwcq
  Created wheel for py-hanspell: filename=py_hanspell-1.1-cp37-none-any.whl size=4854 sha256=1465a3de196e43741f7c303f9369c97213aa916656c656aaf85ecbe107f8b6bd
  Stored in directory: /tmp/pip-ephem-wheel-cache-lcmq5629/wheels/0a/25/d1/e5e96476dbb1c318cc26c992dd493394fe42b0c204b3e65588
Successfully built py-hanspell


In [24]:
from hanspell import spell_checker
from tqdm.notebook import tqdm

data['checked'] = ''
for i in tqdm(range(len(data))):
  data['checked'][i] = spell_checker.check(data['All'][i]).as_dict()['checked'] 
data

,level2,Chief complaint,Onset,Location,All,checked
0,급성 복통,배가 아파요,1일전,명치 부위,배가 아파요. 1일전. 명치 부위,배가 아파요. 1일 전. 명치 부위
1,급성 복통,갑자기 배가 너무 아파요,오늘 새벽,명치에서 시작해 현재 복부 전체에 통증,갑자기 배가 너무 아파요. 오늘 새벽. 명치에서 시작해 현재 복부 전체에 통증,갑자기 배가 너무 아파요. 오늘 새벽. 명치에서 시작해 현재 복부 전체에 통증
2,급성 복통,속이 쓰려요,1달전,명치 부위,속이 쓰려요. 1달전. 명치 부위,속이 쓰려요. 1달 전. 명치 부위
3,급성 복통,속이 쓰려요,2달전,명치부위,속이 쓰려요. 2달전. 명치부위,속이 쓰려요. 2달 전. 명치 부위
4,급성 복통,배가 아파요,1시간전,명치부위,배가 아파요. 1시간전. 명치부위,배가 아파요. 1시간 전. 명치 부위
...,...,...,...,...,...,...
2365,토혈,피를 토했어요,,,피를 토했어요. .,피를 토했어요. .
2366,토혈,피를 토했어요,,,피를 토했어요. .,피를 토했어요. .
2367,토혈,피를 토했어요,,,피를 토했어요. .,피를 토했어요. .
2368,토혈,피 토했어요,,,피 토했어요. .,피 토했어요. .


## 5) Erase stopwords using konlpy

In [25]:
!pip install konlpy
import konlpy.tag
okt = konlpy.tag.Okt() # 객체 생성

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 8.2MB/s 
     |████████████████████████████████| 460kB 39.7MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [26]:
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '을',
             '를', '으로', '자', '에', '와', '하다', '요', '다', '.', ',']

In [27]:
import re

def erase_stopwords(text):
  X_train =[]
  temp_x = okt.morphs(text, stem=True)
  #print(temp_x)
  temp_x = [word for word in temp_x if not word in stopwords]
  temp_x = re.findall(r'\w+', str(temp_x))
  X_train.append(temp_x)
  #print(X_train)
  temp_x = ' '.join(map(str, X_train))
  
  return ' '.join(re.findall(r'\w+', str(temp_x)))

In [28]:
data['checked'] = data.apply(lambda x : erase_stopwords(x['checked']) , axis = 1 )

# 2. Generate feature
https://datascience.stackexchange.com/questions/74688/how-to-use-multiple-text-features-for-nlp-classifier

In [29]:
data_level2 = data.pop('level2')

In [30]:
from sklearn.model_selection import train_test_split

ori_X_train, ori_X_test, ori_y_train, ori_y_test = train_test_split(data, data_level2, random_state = 0)

In [31]:
ori_X_train

,Chief complaint,Onset,Location,All,checked
596,피부 발진,3개월전,피부,피부 발진. 3개월전. 피부,피부 발진 3 개월 전 피부
77,배가 아파요,2일전,우측복부,배가 아파요 . 2일전. 우측복부,배 아프다 2일 전 우측 복부
2156,피를 토했어요,6시간 전,,피를 토했어요. 6시간 전.,피 토 6시간 전
393,얼굴이 노래요,2일전,,얼굴이 노래요. 2일전.,얼굴 노래 2일 전
2180,피 토했어요,5시간 전,,피 토했어요. 5시간 전.,피 토 5시간 전
...,...,...,...,...,...
1033,걸으면 숨이 많이 차요,3달전,,걸으면 숨이 많이 차요. 3달전.,걸다 숨 많이 차요 3 달 전
1731,얼굴이 노래요,한달전,"얼굴, 사지","얼굴이 노래요. 한달전. 얼굴, 사지",얼굴 노래 달 전 얼굴 사지
763,"""소변이 붉어요""",어제 아침,,"""소변이 붉어요"". 어제 아침.",소변 붉다 어제 아침
835,"""자주 소변이 새요""",7년 전부터 조금씩 서서히 발생,,"""자주 소변이 새요"". 7년 전부터 조금씩 서서히 발생.",자주 소변 새 7년 전 부터 조금씩 서서히 발생


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(ori_X_train['checked'])

# for test
X_test_tfidf = vectorizer.transform(ori_X_test['checked'])

In [33]:
data_level2.value_counts()

황달            495
토혈            400
호흡곤란          120
월경이상/월경통      100
급성 복통          95
목 통증/허리 통증     80
관절 통증          75
유방통/유방덩이       70
가슴통증           66
실신             65
붉은색 소변         60
배뇨 이상          60
기침             60
구토             60
두근거림           55
소화불량/만성 복통     55
객혈             50
피부발진           50
핍뇨             50
설사             45
고혈압            43
혈변             40
다뇨증            40
콧물/코막힘         40
소변찔끔증          36
질 분비물          30
변비             30
Name: level2, dtype: int64

# Model

In [34]:
from sklearn.naive_bayes import MultinomialNB # 다중 Naive Bayes 분류기
from sklearn.ensemble import RandomForestClassifier
#from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

param_grid = {
    'max_depth': [45, 50, 60, 70],
    'n_estimators': [20, 50, 100],
    'min_samples_split': [0.01, 0.05, 0.1],
}

clf = RandomForestClassifier(random_state=0, class_weight="balanced")

grid = GridSearchCV(clf, param_grid=param_grid, cv=3, refit=True)

grid.fit(X_train_tfidf, ori_y_train)

print('GridSearchCV best parameters:', grid.best_params_)
print('GridSearchCV best acc: {0:.2f}'.format(grid.best_score_))

estimator = grid.best_estimator_
pred_labels = estimator.predict(X_test_tfidf)

#clf = RandomForestClassifier(max_depth=60, random_state=0)
#clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
#                                 max_depth=30, random_state=0)

#ml_model = nb.fit(X_train_tfidf, ori_y_train) # 학습
#ml_model = clf.fit(X_train_tfidf, ori_y_train) # 학습
#train_pred_labels = ml_model.predict(X_train_tfidf) # 학습된 모델로 예측
#pred_labels = ml_model.predict(X_test_tfidf) # 학습된 모델로 예측

#print('Number of misclassified diseased of train data : {} out of {}'.format((train_pred_labels != ori_y_train).sum(), len(ori_y_train)))
#print('Accuracy train : %.2f' % accuracy_score(ori_y_train, train_pred_labels))

print('Number of misclassified diseased of test data : {} out of {}'.format((pred_labels != ori_y_test).sum(), len(ori_y_test)))
print('Accuracy test : %.2f' % accuracy_score(ori_y_test, pred_labels))

GridSearchCV best parameters: {'max_depth': 60, 'min_samples_split': 0.01, 'n_estimators': 100}
GridSearchCV best acc: 0.97
Number of misclassified diseased of test data : 19 out of 593
Accuracy test : 0.97


In [35]:
#location 제외

#class weight
#GridSearchCV best parameters: {'max_depth': 60, 'min_samples_split': 0.01, 'n_estimators': 50}
#GridSearchCV best acc: 0.97
#Number of misclassified diseased of test data : 33 out of 563
#Accuracy test : 0.94
#top_k_result: 0.9840

In [36]:
test_dummies = pd.get_dummies(ori_y_test)

In [37]:
def top_k_hitrate(predicted_proba, n=2):

  eval_metric = []
  top_k_result = tf.math.top_k(predicted_proba, k=n, sorted=True)
  for i in range(len(top_k_result[1])):
    first = top_k_result[1][i][0]
    second = top_k_result[1][i][1]
#    third = top_k_result[1][i][2]

    if ori_y_test.iloc[i] in (test_dummies.columns[first],
                              test_dummies.columns[second]):
#                              test_dummies.columns[third]):
      eval_metric.append(1)
    else:
      eval_metric.append(0)

  return np.mean(eval_metric)

In [38]:
print("top_k_result: {:.4f}".format(top_k_hitrate(estimator.predict_proba(X_test_tfidf), n=2)))

top_k_result: 0.9949


In [39]:
tf.math.top_k(estimator.predict_proba(X_test_tfidf), k=2)

TopKV2(values=<tf.Tensor: shape=(593, 2), dtype=float64, numpy=
array([[0.97172474, 0.01310454],
       [0.87823167, 0.02846205],
       [0.96368616, 0.03258159],
       ...,
       [1.        , 0.        ],
       [0.77210034, 0.16185442],
       [0.76581544, 0.15882421]])>, indices=<tf.Tensor: shape=(593, 2), dtype=int32, numpy=
array([[22, 17],
       [ 3,  0],
       [24,  7],
       ...,
       [ 2,  0],
       [ 9,  3],
       [18,  0]], dtype=int32)>)

In [40]:
ori_y_test = np.asarray(ori_y_test)
misclassified = np.where(ori_y_test != estimator.predict(X_test_tfidf))

pd.concat([pd.DataFrame(ori_y_test[misclassified], columns=["True value"]),
          pd.DataFrame(estimator.predict(X_test_tfidf)[misclassified], columns=["Predict"]),
           pd.DataFrame(ori_X_test['All'].iloc[misclassified]).reset_index(drop=True)],
          axis=1)

,True value,Predict,All
0,관절 통증,목 통증/허리 통증,허리 아파요. 2년전. 허리
1,유방통/유방덩이,가슴통증,오른쪽 가슴이 아파요. 한달 반 전. 오른쪽
2,급성 복통,소화불량/만성 복통,속이 쓰려요. 1주전. 배꼽주변
3,객혈,토혈,피를 토했어요. 2시간전.
4,배뇨 이상,소변찔끔증,"""소변이 자주 마려워요"". 약 6개월 전부터 서서히 시작됨. 종이컵 1잔 분량/회"
5,급성 복통,관절 통증,갑자기 배가 너무 아파요. 오늘 새벽. 명치에서 시작해 현재 복부 전체에 통증
6,월경이상/월경통,토혈,질에서 피가 나와요. 7시간 전.
7,토혈,객혈,피 토했어요. 2일 전.
8,목 통증/허리 통증,관절 통증,허리 아파요. 3년전. 허리
9,급성 복통,소화불량/만성 복통,속이 쓰려요. 1일전. 배꼽주면


# Save Models

In [41]:
# Save ML model
import pickle

with open('level2_estimator.pkl', 'wb') as file:
    pickle.dump(estimator, file)

with open("level2_dummies.txt", "wb") as fp:
  pickle.dump(pd.get_dummies(a['disease']).columns, fp)

with open("level2_tfidf_vectorizer.pkl", 'wb') as fw:
     pickle.dump(vectorizer, fw)

# For test

In [42]:
import pickle

with open('level2_tfidf_vectorizer.pkl', 'rb') as b:
    tfidf = pickle.load(b)

with open('level2_estimator.pkl', 'rb') as c:
    load_model = pickle.load(c)

with open('level2_dummies.txt', 'rb') as d:
    dummies = pickle.load(d)

In [43]:
test_df = {'Chief complaint': '속이 자주 쓰려요',
           'Onset' : '일주일전',
           'Location': '명치'}

new_data = pd.DataFrame([test_df])
new_data

,Chief complaint,Onset,Location
0,속이 자주 쓰려요,일주일전,명치


In [44]:
for i in range(len(new_data.columns)):
    new_data[new_data.columns[i]] = new_data.apply(lambda x : to_nan(x[new_data.columns[i]]) , axis = 1 )

In [45]:
new_data['All'] = new_data['Chief complaint'] + '. ' + new_data['Onset'] + '. ' + new_data['Location']

In [46]:
new_data['All'] = spell_checker.check(new_data['All']).as_dict()['checked'] 

In [47]:
new_data

,Chief complaint,Onset,Location,All
0,속이 자주 쓰려요,일주일전,명치,속이 자주 쓰려요. 일주일 전. 명치


In [48]:
new_data['All'] = new_data.apply(lambda x : erase_stopwords(x['All']) , axis = 1 )

In [49]:
test_data = tfidf.transform(new_data['All'])

In [50]:
top_k_result = tf.math.top_k(load_model.predict_proba(test_data), k=2, sorted=True)

In [51]:
first = top_k_result[1][0][0], top_k_result.values.numpy()[0][0]
second = top_k_result[1][0][1], top_k_result.values.numpy()[0][1]
#third = top_k_result[1][0][2], top_k_result.values.numpy()[0][2]

In [52]:
first[1] - second[1]

0.2212794658980266

In [53]:
if (first[1] - second[1] < 0.1):
  print(dummies[first[0]],':', round(first[1]*100, 2),"%")
  print(dummies[second[0]],':', round(second[1]*100, 2),"%")
else:
  print(dummies[first[0]],':', round(first[1]*100, 2),"%")

소화불량/만성 복통 : 51.14 %


In [54]:
with open('level2_dummies.txt', 'rb') as d:
    dummies = pickle.load(d)
dummies

Index(['가슴통증', '객혈', '고혈압', '관절 통증', '구토', '급성 복통', '기침', '다뇨증', '두근거림',
       '목 통증/허리 통증', '배뇨 이상', '변비', '붉은색 소변', '설사', '소변찔끔증', '소화불량/만성 복통',
       '실신', '월경이상/월경통', '유방통/유방덩이', '질 분비물', '콧물/코막힘', '토혈', '피부발진', '핍뇨',
       '혈변', '호흡곤란', '황달'],
      dtype='object')